# Encapsulated execution Notebook

This notebook has the methods for finding the COs in a trace a little bit more encapsulated. We can call a testcase, the TraceCreator will load the trace with the necessary information and the we can employ the SampleFinder to find the correct starting points which can then be refined and evaluated.

In [ ]:
from src.refiner import Refiner
from src.trace_creator import TraceImporter
from src.sample_finder import SampleFinder

#uncomment testcases as you wish to try different scenarios mentioned in the paper.
testcase = "stm32f4_tinyaes"           #tested with a decimation factor of 5
#testcase = "stm32f4_HWAES"             #tested with a decimation factor of 1
#testcase = "sha256_external_clock"     #tested with a decimation factor of 15
#testcase = "sha256_internal_clock"     #tested with a decimation factor of 25 (to use less ressources)
#testcase = "mbed_tls_AES"              #tested with a decimation factor of 25 
                                        #This testcase might be too large for your PC

proposed_decimation_factors = {
    "stm32f4_tinyaes":5,
    "stm32f4_HWAES":1,
    "sha256_external_clock":15,
    "sha256_internal_clock":25,
    "mbed_tls_AES":25
    }

decimation_factor = proposed_decimation_factors[testcase]   #only change this if you know what you're doing. 
                        #If the decimation factor is too high, the algorithm will not be able to find the pattern
                        #The samples per clock need to be an integer. Set this variable accordingly.

trace_creator =  TraceImporter(testcase,do_plots=False)
trace_container = trace_creator.trace_container

trace_container.decimate_trace(decimation_factor)
#Plot the imported trace in a decimated fashion:
trace_container.plot_trace(decimate_for_plot_factor=40)

## Finding the COs after we imported the Trace

After sucessfully importing the Trace, we can now start to search for the COs in the trace.
For this we just use the given values we found.

In [ ]:
#Plots also take a lot of time here because they are very large...
do_quality_plot = True #quality q_w(i) plot which is calculated in Algorithm 1
do_main_sub_peak_plot = True # plot the correlation which is used to find the main and sub-peaks in Algorthm 2

print("Info: Starting to find COs in a trace with length " + str(len(trace_container.get_trace())) + ".")

sample_finder = SampleFinder(trace_container,
    exact_clk_cycles=trace_container.known_width_clk_cycles,
    error_margin=0.1,
    allowed_sub_peak_delta=2
    )


found_width, found_peak_idx,char_trace_template = sample_finder.full_auto_find_COs(do_quality_plot=do_quality_plot,
    do_main_sub_peak_plot=do_main_sub_peak_plot,
    avg_round_template=True,
    sad_approach=False,
    use_detrended=False,
    sad_for_autocorr=False)


After finding all the COs we can improve the found indices further by using the refinement of the template

In [ ]:
refiner = Refiner(trace_container)
refined_template = refiner.get_template_with_sad()

trace_container.calculated_start_idx_aes = sample_finder.find_COs_with_template(refined_template)
trace_container.plot_trace(decimate_for_plot_factor=40)


After refining the found indices, we can now look at the accuracy because we know where the COs in this trace are.

In [ ]:
from src.evaluator import Evaluator
print("Refined accuracy:  ---------------------------------------------------------------------")
evaluator_after_refinement = Evaluator(trace_container,do_plots=True)


# Unlimited Refinement

After finding (some of) the correct starting indices, we can further refine the template by using the refinement multiple times until nothing changes anymore or we reach a threshold.

This may not be needed if the results are already good enough. For larger traces this can take a while.

In [ ]:
import numpy as np

for i in range(5): #refine 5 times or until no more improvement:
    print("Refining Round " + str(i+1))
    old_start_idx = trace_container.calculated_start_idx_aes

    refiner = Refiner(trace_container)
    refined_template = refiner.get_template_with_sad(plot_finished_template=True)
    refined_start_idx = sample_finder.find_COs_with_template(refined_template)

    if np.array_equal(old_start_idx,refined_start_idx):
        break
    else:
        trace_container.calculated_start_idx_aes=refined_start_idx

print("Finished refinement after "+str(i+1)+" rounds.")
print("refined accuracy:  ---------------------------------------------------------------------")
evaluator_after_refinement = Evaluator(trace_container,do_plots=True,max_rnd_offset_adj=205)
